In [ ]:
#!/usr/bin/env python
# coding: utf-8

# ==================================================================== #
# author: Yung-Hsin Chen                                               #
# copyright: Copyright 2020, Pneumonia Diagnosis                       #                                                            
# ==================================================================== #

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.applications import VGG16
from keras import layers, models
import json
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True' # for MAC

Using TensorFlow backend.


### $\underline{Loading\;and\;Storing\;Data}$
##### - Classes: pneumonia, normal
##### - Train (228x228x100), Validation (228x228x16), Test (228x228x5)

In [ ]:
imgX = imgY = 224
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.15,
    rotation_range=10,
    zoom_range=0.1,
)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
path = '/content/drive/My Drive/Colab Notebooks/Pneumonia_ML/chest_x_ray'
train_generator = train_datagen.flow_from_directory(
    path + '/data/train',
    target_size = (imgX, imgY),
    batch_size = 50,#5216
    class_mode = "binary",
    shuffle = True
)
validation_generator = val_datagen.flow_from_directory(
    path +'/data/val',
    target_size = (imgX, imgY),
    batch_size = 16,
    class_mode = "binary",
    shuffle = False
)
test_generator = test_datagen.flow_from_directory(
    path +'/data/test',
    target_size = (imgX, imgY),
    batch_size = 5,#624
    class_mode = "binary",
    shuffle = False
)

Found 5226 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### $\underline{Neural\;Network\;Layers}$
##### - Conv2D
##### - Maxpooling2D
##### - Conv2D
##### - Maxpooling2D
##### - Conv2D
##### - Dense (Sigmoid)
##### - Dropout (0.5)
##### - Dense (Sigmoid)

In [ ]:
# pretrain = tf.keras.applications.vgg16.VGG16(weights = 'imagenet', include_top = False, input_tensor = tf.keras.layers.Input(shape=(imgX, imgY, 3)))
# pretrain.trainable = False
conv1 = tf.keras.layers.Conv2D(128, (3, 3), activation='sigmoid', padding = 'same')
# conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='sigmoid', padding = 'same')
pool1 = tf.keras.layers.MaxPooling2D((2, 2))
conv2 = tf.keras.layers.Conv2D(256, (3, 3), activation='sigmoid', padding = 'same')
pool2 = tf.keras.layers.MaxPooling2D((2, 2))
conv3 = tf.keras.layers.Conv2D(256, (3, 3), activation='sigmoid', padding = 'same')
flatten = tf.keras.layers.Flatten()
dense1 = tf.keras.layers.Dense(256, activation='sigmoid')
drop1 = tf.keras.layers.Dropout(0.5)
dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
model = tf.keras.Sequential([tf.keras.layers.Input(shape=(imgX, imgY, 3)), conv1, pool1, conv2, pool2, conv3, flatten, dense1, drop1, dense2])

In [ ]:
conv1.trainable = False
pool1.trainable = False
conv2.trainable = False
pool2.trainable = False
conv3.trainable = False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 128)     3584      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 256)     295168    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       590080    
_________________________________________________________________
flatten (Flatten)            (None, 802816)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               2

### $\underline{Training}$

In [ ]:
print("START COMPILING")
model.compile(loss=tf.keras.losses.binary_crossentropy, optimizer=tf.keras.optimizers.RMSprop(), metrics=[tf.keras.metrics.binary_crossentropy,'accuracy'])
print("START FITTING")
history = model.fit(train_generator, epochs = 3, validation_data = test_generator)
score = model.evaluate(test_generator, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[2])

START COMPILING
START FITTING
Epoch 1/3
105/105 [==============================] - 1459s 14s/step - loss: 1.3704 - binary_crossentropy: 1.3704 - accuracy: 0.7413 - val_loss: 0.7056 - val_binary_crossentropy: 0.7056 - val_accuracy: 0.6250
Epoch 2/3
105/105 [==============================] - 1458s 14s/step - loss: 0.5910 - binary_crossentropy: 0.5910 - accuracy: 0.7392 - val_loss: 0.7001 - val_binary_crossentropy: 0.7001 - val_accuracy: 0.6250
Epoch 3/3
105/105 [==============================] - 1460s 14s/step - loss: 0.5887 - binary_crossentropy: 0.5887 - accuracy: 0.7409 - val_loss: 0.6861 - val_binary_crossentropy: 0.6861 - val_accuracy: 0.6250
Test loss: 0.6861472725868225
Test accuracy: 0.625


### $\underline{Saving\;History}$

In [ ]:
history_file = pd.DataFrame(history.history)
path = '/content/drive/My Drive/Colab Notebooks/Pneumonia_ML/chest_x_ray'
with open('path + /history2.json', 'w') as fh:
    history_file.to_json(fh)
history_json                 = pd.read_json('history.json')
accuracy                     = history_json['accuracy']
val_accuracy                 = history_json['val_accuracy']
loss                         = history_json['loss']
val_loss                     = history_json['val_loss']
categorical_crossentropy     = history_json['categorical_crossentropy']
val_categorical_crossentropy = history_json['val_categorical_crossentropy']

### $\underline{Plot\;of\;accuracy}$

In [ ]:
plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('model-images/accuracy.png',dpi=160);
plt.show()